In [58]:
import json
import pandas as pd
result_path = '../table-union/results-cl/top_20_results.json'



with open(result_path) as f:
    results = json.load(f)
    df = pd.DataFrame(results)

df


,Table name,Candidate column,Ground truth column,Top k columns
0,Krug,Tumor.Stage,ajcc_pathologic_stage,"[figo_stage, uicc_clinical_stage, ajcc_patholo..."
1,Krug,Age.in.Month,age_at_diagnosis,"[age_at_diagnosis, age_at_index, relationship_..."
2,Krug,Gender,gender,"[gender, relationship_gender, pregnant_at_diag..."
3,Krug,Ethnicity,ethnicity,"[race, ethnicity, eye_color, country_of_reside..."
4,Clark,Tumor/Normal,tissue_type,"[tissue_type, non_nodal_tumor_deposits, select..."
...,...,...,...,...
121,Dou,Ethnicity,ethnicity,"[race, ethnicity, variant_origin, country_of_b..."
122,Dou,Gender,gender,"[gender, relationship_gender, inpc_histologic_..."
123,Dou,Tumor_Site,tissue_or_organ_of_origin,"[margins_involved_site, supratentorial_localiz..."
124,Dou,Tumor_Focality,tumor_focality,"[tumor_focality, non_nodal_tumor_deposits, tum..."


In [59]:
gdc_path = '../table-union/data/gdc_schema_extracted.csv'
gdc_df = pd.read_csv(gdc_path)
gdc_cols = gdc_df['column name'].unique()

In [60]:
df = df[~df['Candidate column'].str.lower().isin(gdc_cols)]
df


,Table name,Candidate column,Ground truth column,Top k columns
0,Krug,Tumor.Stage,ajcc_pathologic_stage,"[figo_stage, uicc_clinical_stage, ajcc_patholo..."
1,Krug,Age.in.Month,age_at_diagnosis,"[age_at_diagnosis, age_at_index, relationship_..."
4,Clark,Tumor/Normal,tissue_type,"[tissue_type, non_nodal_tumor_deposits, select..."
6,Clark,Age,age_at_diagnosis,"[age_at_onset, age_at_index, relationship_age_..."
9,Clark,Ethnicity_Self_Identify,ethnicity,"[race, ethnicity, demographics, country_of_bir..."
...,...,...,...,...
115,Dou,Path_Stage_Dist_Mets-pM,ajcc_pathologic_m,"[ajcc_clinical_m, ajcc_pathologic_m, uicc_clin..."
116,Dou,tumor_Stage-Pathological,ajcc_pathologic_stage,"[ensat_pathologic_stage, inss_stage, iss_stage..."
119,Dou,Age,age_at_diagnosis,"[age_at_index, relationship_age_at_diagnosis, ..."
123,Dou,Tumor_Site,tissue_or_organ_of_origin,"[margins_involved_site, supratentorial_localiz..."


In [61]:
interested_columns  = set()
interested_columns.update(df['Ground truth column'].unique()) 



interested_columns.update([item for sublist in df['Top k columns'] for item in sublist])
len(interested_columns)



454

In [62]:
filtered_gdc_df = gdc_df[gdc_df['column name'].isin(interested_columns)]
filtered_gdc_df

filtered_gdc_df


,column name,column type,column values
1,aa_change,string,Not specified
4,adapter_sequence,string,Not specified
5,additional_pathology_findings,Not specified,"Asbestos bodies, Bilateral ovaries with endome..."
8,adverse_event_grade,Not specified,"Grade 1, Grade 2, Grade 3, Grade 4, Grade 5"
9,age_at_diagnosis,number,Not specified
...,...,...,...
725,workflow_link,string,Not specified
729,year_of_birth,Not specified,Not specified
730,year_of_death,integer,Not specified
733,years_smoked,number,Not specified


In [63]:

specified_values_gdc = filtered_gdc_df[filtered_gdc_df['column values'] != 'Not specified']
non_specified_values_gdc = filtered_gdc_df[filtered_gdc_df['column values'] == 'Not specified']

print(len(specified_values_gdc))
print(len(non_specified_values_gdc))
total_len = len(specified_values_gdc) + len(non_specified_values_gdc)
print(total_len)

248
206
454


In [64]:
specified_values_gdc

,column name,column type,column values
5,additional_pathology_findings,Not specified,"Asbestos bodies, Bilateral ovaries with endome..."
8,adverse_event_grade,Not specified,"Grade 1, Grade 2, Grade 3, Grade 4, Grade 5"
16,ajcc_clinical_m,Not specified,"M0, M1, M1a, M1b, M1c, MX, Not Allowed To Coll..."
17,ajcc_clinical_n,Not specified,"N0 (i+), N0 (mol-), N1, N1a, N1b, N1bI, N1bIV,..."
18,ajcc_clinical_stage,Not specified,"Stage 0, Stage IA2, Stage IB2, Stage IIA, Stag..."
...,...,...,...
720,wgs_coverage,Not specified,"0x-10x, 10x-25x, 150x+, 25x-150x, Not Applicab..."
721,who_cns_grade,Not specified,"Grade I, Grade II, Grade III, Grade IV, Grade ..."
722,who_nte_grade,Not specified,"G1, G2, G3, GX, Not Reported, Unknown"
723,wilms_tumor_histologic_subtype,Not specified,"Favorable, Not Reported, Unfavorable, Unknown"


In [65]:
non_specified_values_gdc

,column name,column type,column values
1,aa_change,string,Not specified
4,adapter_sequence,string,Not specified
9,age_at_diagnosis,number,Not specified
10,age_at_index,Not specified,Not specified
11,age_at_last_exposure,integer,Not specified
...,...,...,...
717,weiss_assessment_findings,array,Not specified
725,workflow_link,string,Not specified
729,year_of_birth,Not specified,Not specified
730,year_of_death,integer,Not specified
